# Nova Corrente Storytelling Demo

Este notebook demonstra como consumir os artefatos gerados por `scripts.storytelling.build_storytelling_assets` para alimentar dashboards e narrativas de negócios.


In [ ]:
from pathlib import Path
import pandas as pd

from data_sources.storytelling_loader import (
    load_timeseries,
    load_prescriptions,
    get_kpi_snapshot,
    get_alerts_by_status,
    get_family_timeseries,
)

root = Path("../../data/outputs/nova_corrente/storytelling")

kpis = get_kpi_snapshot(root)
alerts = get_alerts_by_status(root, statuses=["critical"])
family_ts = get_family_timeseries("ABRASIVOS", root)

kpis, alerts.head(), family_ts.head()


In [ ]:
import matplotlib.pyplot as plt

series_key = family_ts["series_key"].iloc[0]
series_df = family_ts[family_ts["series_key"] == series_key].copy()

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(series_df["ds"], series_df["actual_qty"], label="Histórico", marker="o", linewidth=1)
ax.plot(series_df["ds"], series_df["forecast_qty"], label="Forecast", linestyle="--")
ax.fill_between(
    series_df["ds"],
    series_df.get("lower", series_df["forecast_qty"]),
    series_df.get("upper", series_df["forecast_qty"]),
    color="tab:blue",
    alpha=0.2,
    label="Intervalo",
)
ax.set_title(f"Série {series_key}")
ax.set_xlabel("Data")
ax.set_ylabel("Consumo / Previsão")
ax.legend()
plt.tight_layout()
plt.show()


### Principais Alertas Críticos

Esta tabela pode alimentar o painel de comando prescritivo, destacando pontos de ruptura iminente.


In [ ]:
alerts[['series_key', 'familia', 'region', 'reorder_point', 'safety_stock', 'current_stock', 'days_to_rupture', 'stock_status']].head(10)


In [ ]:
output_dir = Path("../visualizations")
output_dir.mkdir(parents=True, exist_ok=True)

fig_path = output_dir / "storytelling_forecast_sample.png"
table_path = output_dir / "storytelling_alerts_top10.csv"

fig.savefig(fig_path, dpi=150)
alerts.head(10).to_csv(table_path, index=False)

fig_path, table_path
